# Machine Learning Capstone Project

## CNN Classifier

### 1. Import required packages

In [1]:
from helper_functions import unpickle, extract_data, load_data
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from time import time

Using TensorFlow backend.


### 2. Extract CIFAR 15

In [9]:
(X_train_orig, y_train_orig), (X_test, y_test) = load_data('cifar-15-python', 'fine')
print('Training Data Shape  :', X_train_orig.shape)
print('Training Label Shape :', y_train_orig.shape)
print('Testing Data Shape   :', X_test.shape)
print('Testing Label Shape  :', y_test.shape)

Training Data Shape  : (7500, 32, 32, 3)
Training Label Shape : (7500, 1)
Testing Data Shape   : (1500, 32, 32, 3)
Testing Label Shape  : (1500, 1)


#### 2.1. Split training datset into training and validation set

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, 
                                                    test_size=0.20, random_state=10)
print('Training Data Shape    :', X_train.shape)
print('Training Label Shape   :', y_train.shape)
print('Validation Data Shape  :', X_val.shape)
print('Validation Label Shape :', y_val.shape)

Training Data Shape    : (6000, 32, 32, 3)
Training Label Shape   : (6000, 1)
Validation Data Shape  : (1500, 32, 32, 3)
Validation Label Shape : (1500, 1)


### 3. Preprocessing Data

In [11]:
def convertToGray(input_dataset, output_dataset):
    i = 0
    for img in input_dataset:
        output_dataset[i] = np.reshape(np.dot(img[...,:3], [0.299, 0.587, 0.114]), (img.shape[0], img.shape[1], 1))
        i += 1    

In [12]:
X_train_gray = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_val_gray = np.zeros((X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))
X_test_gray = np.zeros((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

# Convert to gray
convertToGray(X_train, X_train_gray)
convertToGray(X_val, X_val_gray)
convertToGray(X_test, X_test_gray)

# Normalize the data
X_train_gray = (X_train_gray - 127.5)/255.0
X_val_gray = (X_val_gray - 127.5)/255.0
X_test_gray = (X_test_gray - 127.5)/255.0

# Reshape image label into 1D array
y_train = np.ravel(y_train)
y_val = np.ravel(y_val)
y_test = np.ravel(y_test)

print('Training Data Shape    :', X_train_gray.shape)
print('Training Label Shape   :', y_train.shape)
print('Validation Data Shape  :', X_val_gray.shape)
print('Validation Label Shape :', y_val.shape)
print('Testing Data Shape     :', X_test_gray.shape)
print('Testing Label Shape    :', y_test.shape)

Training Data Shape    : (6000, 32, 32, 1)
Training Label Shape   : (6000,)
Validation Data Shape  : (1500, 32, 32, 1)
Validation Label Shape : (1500,)
Testing Data Shape     : (1500, 32, 32, 1)
Testing Label Shape    : (1500,)


In [13]:
# Variables 
BATCH_SIZE = 32 
NUM_CLASSES = 15
EPOCHS = 5

In [16]:
y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
y_val = np_utils.to_categorical(y_val, NUM_CLASSES)
y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

In [25]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

input_shape = (32,32,1)

model = Sequential()

# Filter controls the depth of the layer. Stride 1 and 'same' padding will keep layer size(height and width) same. 
# MaxPooling will reduce the size. 
model.add(Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.25))

# At the end of convolution layers, the layer will contain informormation such as Are there wheels in the image?, 
# eyes?, legs?. So the spatial information is converted to content information.
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 16)        80        
_________________________________________________________________
flatten_3 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 15)                245775    
Total params: 245,855.0
Trainable params: 245,855.0
Non-trainable params: 0.0
_________________________________________________________________


In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
t1 = time()
history = model.fit(X_train_gray, y_train, 
          validation_data=(X_val_gray, y_val),
          epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpointer], verbose=1)
# list all data in history
#print(history.history.keys())

# summarize history for accuracy and loss

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
ax1.set_title('model accuracy')
ax1.set_ylabel('accuracy')
ax1.set_xlabel('epoch')
ax1.legend(['train', 'val'], loc='upper left')
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('model loss')
ax2.set_ylabel('loss')
ax2.set_xlabel('epoch')
ax2.legend(['train', 'val'], loc='upper left')
f.tight_layout()
plt.show()
dt = time() - t1
print('Time to train CNN is', dt, 'seconds.' )

Train on 6000 samples, validate on 1500 samples
Epoch 1/10
6000/6000 [==============================] - 12s - loss: 2.0454 - acc: 0.2960 - val_loss: 1.8739 - val_acc: 0.3513
Epoch 2/10
4064/6000 [===================>..........] - ETA: 3s - loss: 1.8609 - acc: 0.3593 

### 5. Accuracy of the classifier on test set

In [6]:
score = model.evaluate(X_test_gray, y_test)
#print('Accuracy of the CNN classifier on test set is', score*100, '%')
print('Accuracy of the CNN classifier on test set is', score[1]*100, '%')

Accuracy of the SVM classifier is 25.7333333333 %
